###  Shapes with Assertions (Java) [8 points + 4 bonus points]

The [Composite](https://en.wikipedia.org/wiki/Composite_pattern) design pattern is one of the [original 23 object-oriented design patterns](https://en.wikipedia.org/wiki/Design_Patterns) ([PDF](http://www.uml.org.cn/c++/pdf/designpatterns.pdf)). Here, it is applied to *geometric shapes*, like those used in drawing editors. A shape is a line, a circle, a rectangle, etc., but shapes can also be grouped into larger shapes. A grouped shape is considered to be a shape that can again be grouped. In the design pattern terminology, class `Shape` is the _Component_, classes `Line` and `Rectangle` are _Leafs_, and class `Group` is the _Composite_. Class `Shape` is *abstract*, meaning no objects can be created; it can be extended, and objects of class `Shape` can be declared.

Here, all shapes have a *bounding box*, the smallest rectangle that encloses all the contained shapes. We consider that a group *owns* all the elements it contains, i.e. group elements must not be accessed through references from outside a group (even though Java does not enforce this), so the invariant of a group is allowed to be a *multi-object invariant* over the elements. The task is to extend the code below:

In [1]:
%%writefile shapes.java
import java.util.ArrayList;
import java.util.List;

abstract class Shape {
    int x0, y0, x1, y1; // (x0, y0) lower left, (x1, y1) upper right
    boolean shapeInvariantOK() {
        ...
    }
    Shape(int x0, int y0, int x1, int y1) {
        this.x0 = x0; this.y0 = y0; this.x1 = x1; this.y1= y1;
    }
    void move(int dx, int dy) {
        x0 += dx; y0 += dy; x1 += dx; y1 += dy;
    }
    Rectangle boundingBox () {
        return new Rectangle(x0, y0, x1 - x0, y1 - y0);
    }
}
class Line extends Shape {
    int x, y, u, v; // from (x, y) to (u, v)
    boolean lineInvariantOK() {
        ...
    }
    Line(int x, int y, int u, int v) {
        super(x < u ? x : u, y < v ? y : v,
              x < u ? u : x, y < v ? v : y);
        this.x = x; this.y = y; this.u = u; this.v = v;
    }
    void move(int dx, int dy) {
        super.move(dx, dy);
        x += dx; y += dy; u += dx; v += dy;
    }
}
class Rectangle extends Shape {
    int x, y, w, h;
    boolean rectangleInvariantOK() {
        ...
    }
    Rectangle(int x, int y, int w, int h) {
        super(x, y, x + w, y + h);
        this.x = x; this.y = y; this.w = w; this.h = h;
    }
    void move(int dx, int dy) {
        super.move(dx, dy); x += dx; y += dy;
    }
}
class Group extends Shape {
    List<Shape> elts = new ArrayList<Shape>();
    boolean groupInvariantOK() {
        ...
    }
    Group(Shape s) {
        super(s.x0, s.y0, s.x1, s.y1);
        elts.add(s);
    }
    void move(int dx, int dy) {
        super.move(dx, dy);
        for (Shape s: elts) s.move(dx, dy);
    }
    void add(Shape s) {
        elts.add(s);
        if (s.x0 < x0) x0 = s.x0;
        if (s.y0 < y0) y0 = s.y0;
        if (x1 < s.x1) x1 = s.x1;
        if (y1 < s.y1) y1 = s.y1;
    }
}
class TestShapes {
    public static void main(String[] args) {
        Shape r = new Rectangle(1, 1, 5, 5);
        Shape l = new Line(0, 0, 4, 4);
        Group g = new Group(r);
        g.add(l);
        Rectangle bb = g.boundingBox();
        System.out.println(g.x0);
        System.out.println(g.x1);
        System.out.println(g.y0);
        System.out.println(g.y1);
        g.move(10, 10);
        System.out.println(g.x0);
        System.out.println(g.x1);
        System.out.println(g.y0);
        System.out.println(g.y1);
        // r.move(3, 3); // BAD, breaks invariant of g
        // l.move(-1, -1); // BAD, breaks invariant of g
        assert g.groupInvariantOK();
    }
}

Writing shapes.java


Copy the above code to the cell below:
- Add to `Shape` checks for the invariant: `x0 ≤ x1` and `y0 ≤ y1`.
- Add to `Shape` sufficient checks for method preconditions to ensure the class invariant.
- Add to `Line` checks for the invariant: `x ≠ u` or `y ≠ v`, `x0 = min(x, u)`, `y0 = min(y, v)`, `x1 = max(x, u)`, `y1 = max(y, v)`; additionally, the invariant of `Shape` has to hold.
- Add to `Line` sufficient checks for method preconditions to ensure the class invariant.
- Add to `Rectangle` checks for the invariant: `0 < w`, `0 < h`, `x0 = x`, `y0 = y`, `x1 = x + w`, `y1 = y + h`; additionally, the invariant of `Shape` has to hold.
- Add to `Rectangle` sufficient checks for method preconditions to ensure the class invariant.
- Add to `Group` checks for the invariant: all inserted `Shape` objects must not be `null`, it has to contain at least one `Shape` object, `x0` is the smallest x coordinate of the bounding boxes of the contained shapes, `y0` is the smallest y coordinate, `x1` is the largest x coordinate, and `y1` is the largest y coordinate; additionally, the invariant of `Shape` has to hold.
- Add to `Group` sufficient checks for method preconditions to ensure the class invariant.

You can add test cases as you like; they are not being graded.

In [35]:
%%writefile shapes.java
import java.util.ArrayList;
import java.util.List;

abstract class Shape {
    int x0, y0, x1, y1; // (x0, y0) lower left, (x1, y1) upper right
    boolean shapeInvariantOK() {
        return x0 <= x1 && y0 <= y1;
    }
    Shape(int x0, int y0, int x1, int y1) {
        this.x0 = x0; this.y0 = y0; this.x1 = x1; this.y1= y1;
        assert shapeInvariantOK();
    }
    void move(int dx, int dy) {
        x0 += dx; y0 += dy; x1 += dx; y1 += dy;
        assert shapeInvariantOK();
    }
    Rectangle boundingBox () {
        return new Rectangle(x0, y0, x1 - x0, y1 - y0);
    }
}
class Line extends Shape {
    int x, y, u, v; // from (x, y) to (u, v)
    boolean lineInvariantOK() {
        return x != u && y != v && x0 == Math.min(x,u) && y0 == Math.min(y,v) && x1 == Math.max(x,u) && y1 == Math.max(y,v) && super.shapeInvariantOK();
    }
    Line(int x, int y, int u, int v) {
        super(x < u ? x : u, y < v ? y : v,
              x < u ? u : x, y < v ? v : y);
        this.x = x; this.y = y; this.u = u; this.v = v;
        assert lineInvariantOK();
    }
    void move(int dx, int dy) {
        super.move(dx, dy);
        x += dx; y += dy; u += dx; v += dy;
        assert lineInvariantOK();
    }
}
class Rectangle extends Shape {
    int x, y, w, h;
    boolean rectangleInvariantOK() {
        return w > 0 && h > 0 && x0 == x && y0 == y && x1 == x+w && y1 == y+h && super.shapeInvariantOK();
    }
    Rectangle(int x, int y, int w, int h) {
        super(x, y, x + w, y + h);
        this.x = x; this.y = y; this.w = w; this.h = h;
        assert rectangleInvariantOK();
    }
    void move(int dx, int dy) {
        super.move(dx, dy); x += dx; y += dy;
        assert rectangleInvariantOK();
    }
}
class Group extends Shape {
    List<Shape> elts = new ArrayList<Shape>();
    boolean groupInvariantOK() {
        if (elts.isEmpty()) {
            return false;
        }
        int minX = 999999999, minY = 99999999, maxX = -999999999, maxY = -999999999;
        for (Shape s : elts) {
            if (s != null) {
                if (s.x0 < minX) minX = s.x0;
                if (s.y0 < minY) minY = s.y0;
                if (s.x1 > maxX) maxX = s.x1;
                if (s.y1 > maxY) maxY = s.y1;
            } else {
                return false;
            }
        }
        return x0 == minX && y0 == minY && x1 == maxX && y1 == maxY && super.shapeInvariantOK();
    }
    Group(Shape s) {
        super(s.x0, s.y0, s.x1, s.y1);
        elts.add(s);
        groupInvariantOK();
    }
    void move(int dx, int dy) {
        super.move(dx, dy);
        for (Shape s: elts) s.move(dx, dy);
        groupInvariantOK();
    }
    void add(Shape s) {
        elts.add(s);
        if (s.x0 < x0) x0 = s.x0;
        if (s.y0 < y0) y0 = s.y0;
        if (x1 < s.x1) x1 = s.x1;
        if (y1 < s.y1) y1 = s.y1;
        groupInvariantOK();
    }
}
class TestShapes {
    public static void main(String[] args) {
        Shape r = new Rectangle(1, 1, 5, 5);
        Shape l = new Line(0, 0, 4, 4);
        Group g = new Group(r);
        g.add(l);
        Rectangle bb = g.boundingBox();
        System.out.println(g.x0);
        System.out.println(g.x1);
        System.out.println(g.y0);
        System.out.println(g.y1);
        g.move(10, 10);
        System.out.println(g.x0);
        System.out.println(g.x1);
        System.out.println(g.y0);
        System.out.println(g.y1);
        //r.move(3, 3); // BAD, breaks invariant of g
        //l.move(-1, -1); // BAD, breaks invariant of g
        assert g.groupInvariantOK();
    }
}

Overwriting shapes.java


In [36]:
!javac -Xlint shapes.java

In [37]:
!java -ea TestShapes

0
6
0
6
10
16
10
16


*Bonus Question:* by uncommenting `r.move(3, 3)`, the invariant of `g` will break: all elements of `g` are supposed to be accessed only through `g`, but here we keep a reference to one element and use that. (Note that in Rust, the call `g  = new Group(r)` would transfer ownership and `r` would no longer be accessible; the compiler would forbid the two BAD lines). How can you modify the design in Java to prevent that from happening? Give the modified Java implementation below. You can modify classes `Shape`, `Line`, `Rectangle`, and `Group`. However, if a new shape would be added, say `Circle` with a center and radius, then all existing classes should not need to be modified. Also, your solution has to work with nested groups. Uncomment the two BAD lines; you can add more test cases. Document in comments the idea of your solution. [4 bonus points]

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()